# Prototype the quadrotor and DL

Playground for all code related stuff

In [1]:
# Importing the dependencies
import gym
import numpy as np
from gym import spaces
from scipy.integrate import solve_ivp

In [2]:
# 1. Set values for non linear model
# a. Moment of inertia
I_xx = 0.055
I_yy = 0.055
I_zz = 0.11

# b. Force constants
K_f = 0.119
K_t = 0.0036

# c. 3DOF body specifications
m_hover = 2.85
m_prop = m_hover / 4
g = 9.81
L = 7.75*0.0254 # Distance between pivot and motor

# d. Torque related constants for linear model
Kt_m = 0.0182 # Motor torque constant
Jm = 1.91e-6 # Motor moment of inertia
# i. Equivalent Moment of Inertia of each Propeller Section (kg.m^2)
Jeq_prop = Jm + m_prop*(L**2)
# ii. Equivalent Moment of Inertia about each Axis (kg.m^2)
Jp = 2*Jeq_prop
Jy = 4*Jeq_prop
Jr = 2*Jeq_prop

In [3]:
# 2. Linear model
A_linear = np.array([[0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1], \
                    [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
B_linear = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], \
                    [-K_t/Jy, -K_t/Jy, K_t/Jy, K_t/Jy], [L*K_f/Jp, -L*K_f/Jp, 0, 0], [0, 0, L*K_f/Jr, -L*K_f/Jr]])
C_linear = np.eye(3)
C_linear = np.hstack((C_linear, np.zeros_like(C_linear)))
D_linear = np.zeros((3, 4))

In [4]:
print(f"Dimensions of A: {A_linear.shape}\nDimensions of B: {B_linear.shape}\nDimension of C: {C_linear.shape}\nDimensions of D: {D_linear.shape}")

Dimensions of A: (6, 6)
Dimensions of B: (6, 4)
Dimension of C: (3, 6)
Dimensions of D: (3, 4)


In [5]:
# check for dimension check
u = np.array([4, 5, 2, 6])
vec2 = B_linear@u

x = np.array([-15, 6, 2, -4, 5, 6])
vec1 = A_linear@x

vec1 + vec2

array([-4.        ,  5.        ,  6.        , -0.03259543, -0.42419606,
       -1.69678426])

In [6]:
# QR methods
R = np.eye(4)*0.01
R

array([[0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.01, 0.  , 0.  ],
       [0.  , 0.  , 0.01, 0.  ],
       [0.  , 0.  , 0.  , 0.01]])

In [10]:
Q = np.array([[500, 0, 0, 0, 0, 0], [0, 350, 0, 0, 0, 0], [0, 0, 350, 0, 0, 0],\
                    [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0], [0, 0, 0, 0, 0, 20]])

Q.shape

(6, 6)

## Check for integration methods

In [7]:
# differential equation
def linear_model(t, x, u1, u2, u3, u4):
    """
    Evaluate linear model using the state space form
    ẋ = Ax + Bu
    """
    u = np.array([u1, u2, u3, u4])
    dxdt = A_linear@x + B_linear@u
    return dxdt

In [8]:
# Check for work
timerange = (0.5, 0.502)

# Specify the angles
p = 15*np.pi/180
r = -3*np.pi/180
y = 20*np.pi/180
pdot = 30*np.pi/180
rdot = 20*np.pi/180
ydot = 10*np.pi/180

# Input voltages
u = tuple(np.array([5, 9, 10, 15]))

# state vector
x = np.array([y, p, r, ydot, pdot, rdot])

# get the solution
solution = solve_ivp(linear_model, timerange, x, args=u)

# Next state
solution.y[:, -1]


array([ 0.34941563,  0.26284319, -0.05166599,  0.17525002,  0.52020521,
        0.34482389])

In [9]:
u1, u2, u3, u4 = np.array([5, 9, 10, 15])
u1

5

In [12]:
u = np.array(u)

In [23]:
# Check for rewards calculation

reward = x.T@Q@x
reward = reward + u.T@R@u
type(reward)

numpy.float64

In [14]:
# Check for element wise comparison
u <= np.array([2, 2, 2, 2])

array([False, False, False, False])

In [17]:
ang_rate_max = 60*np.pi/180
p_max = 37.5*np.pi/180
r_max = 37.5*np.pi/180
y_min = 0
y_max = 2*np.pi
max_arr = np.array([p_max, r_max, y_max, ang_rate_max, ang_rate_max, ang_rate_max])
x[3:] >= max_arr[3:]

array([False, False, False])

In [21]:
done = np.any(u <= np.array([2, 2, 2, 2])) or np.any(x[3:] >= max_arr[3:])
done

False

In [24]:
rest = np.zeros_like(x)
rest

array([0., 0., 0., 0., 0., 0.])

In [27]:
x

array([ 0.34906585,  0.26179939, -0.05235988,  0.17453293,  0.52359878,
        0.34906585])

In [28]:
for angle in x[:3]:
    if angle > np.pi:
        angle -= 2*np.pi
    if angle < -np.pi:
        angle += 2*np.pi


In [30]:
x[:3]

array([ 0.34906585,  0.26179939, -0.05235988])

In [29]:
x

array([ 0.34906585,  0.26179939, -0.05235988,  0.17453293,  0.52359878,
        0.34906585])